# creating_database.ipynb
### Author: Alexander X. Gonzalez-Torres
**October 24, 2024**

This Jupyter Notebook documents my process to combine our raw dataset, in its eponymous subdirectory,into an usable database for the project.    



In [1]:
## LIBRARIES USED ##

import pandas as pd
import sqlite3


**STEP 1: Trim WDICSV.csv down to countries by eliminating regions**


In [2]:
# Load the CSV into a DataFrame
wdi_csv = pd.read_csv("raw_dataset/WDICSV.csv")

# Find the index of the first occurrence of "Afghanistan" in the 'Country Name' column
afghanistan_index = wdi_csv[wdi_csv['Country Name'] == "Afghanistan"].index[0]

# Keep all rows starting from the index of "Afghanistan"
trimmed_df = wdi_csv.loc[afghanistan_index:]

# Check the result
print("Numbers of rows expected after trimming: 322896.")
print(f"Number of rows in DataFrame: {len(trimmed_df)}")

# Download DataFrame as .csv 
trimmed_df.to_csv("world_bank_processed.csv", index=False)

Numbers of rows expected after trimming: 322896.
Number of rows in DataFrame: 322896


**STEP 2: Standardize dates in EM-DAT csv (YYYY)**

In [3]:
# Load CSV into DataFrame 

emdat_csv = pd.read_csv("raw_dataset/public_emdat_custom_request_2024-10-11_b70b4036-b2bb-48b2-87db-19b38cd8140f.csv")

# Extract the first 4 characters (which correspond to the year)
emdat_csv['Year'] = emdat_csv['DisNo.'].str[:4]

# Convert the extracted year to integer
emdat_csv['Year'] = emdat_csv['Year'].astype(int)

# Place Year column next to DisNo. 
cols = emdat_csv.columns.tolist()  
disno_index = cols.index('DisNo.')  
cols.insert(disno_index + 1, cols.pop(cols.index('Year')))  
emdat_csv = emdat_csv[cols]  

# Check emdat_csv 
emdat_csv.head()

# Download DataFrame as .csv 
emdat_csv.to_csv("emdat_processed.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'raw_dataset/public_emdat_custom_request_2024-10-11_b70b4036-b2bb-48b2-87db-19b38cd8140f.csv'

**TENTATIVE, STEP 3: Turn WDI Indicators into columns and years into rows**

In [ ]:
# Step 1: Identify the columns that represent years (which should be numeric)
year_columns = [col for col in trimmed_df.columns if col.isdigit()]

# Step 2: Melt the DataFrame, ensuring that only year columns are melted
trimmed_df_melted = pd.melt(
    trimmed_df,
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],  
    value_vars=year_columns,  
    var_name='Year',
    value_name='Value'
)

# Convert the 'Year' column to integers (now we know it only contains year values)
trimmed_df_melted['Year'] = trimmed_df_melted['Year'].astype(int)

# Convert 'Value' column to numeric (this will turn non-numeric values into NaN)
trimmed_df_melted['Value'] = pd.to_numeric(trimmed_df_melted['Value'], errors='coerce')

# Step 3: Pivot the DataFrame so each indicator becomes its own column
trimmed_df_pivoted = trimmed_df_melted.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value',
    aggfunc='first'  
)

# Reset the index to make 'Country Name', 'Country Code', and 'Year' normal columns
trimmed_df_pivoted.reset_index(inplace=True)

# The DataFrame should now have 'Country Name', 'Country Code', 'Year' as the first three columns
# and each 'Indicator Name' as its own column
trimmed_df_pivoted.to_csv("world_bank_long.csv", index=False)

NOTE: Beyond this I'm stuck, we'll probably want to write a script that automates the time-series analysis. I don't think that a single CSV file would be most productive. 

**STEP 4: NEW IDEA, SQL**

In [29]:
## Let's make a project.db


# Load the wide-format economic CSV
econ_wide_df = pd.read_csv('./dataset_processed/world_bank_processed.csv')

# Reshape the economic data from wide to long format
econ_long_df = pd.melt(econ_wide_df,
                       id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                       var_name='Year',  # This will create the 'Year' column
                       value_name='Indicator Value')  # The value of each year

# Convert the 'Year' column to an integer
econ_long_df['Year'] = econ_long_df['Year'].astype(int)

# Rename the columns in the DataFrame to match the table schema
econ_long_df.rename(columns={'Country Name': 'CountryName', 'Country Code': 'CountryCode', 'Indicator Name': 'IndicatorName', 'Indicator Value': 'IndicatorValue'}, inplace=True)

#Drop the 'Indicator Code' column since it's not part of the SQLite schema
econ_long_df.drop(columns=['Indicator Code'], inplace=True)

#Connect to SQLite database
conn = sqlite3.connect('project.db')

# Create the WDI table
create_economic_table = """
CREATE TABLE IF NOT EXISTS WDI (
    CountryCode TEXT,
    CountryName TEXT,
    Year INTEGER,
    IndicatorName TEXT,
    IndicatorValue REAL,
    PRIMARY KEY (CountryCode, Year, IndicatorName)
);
"""
conn.execute(create_economic_table)

# Insert the reshaped data into SQLite
econ_long_df.to_sql('WDI', conn, if_exists='append', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data successfully inserted into the SQLite database!") 

Data successfully inserted into the SQLite database!


In [30]:
## Test project.db integrity 

conn = sqlite3.connect('project.db')

# Query to count the number of rows in the economics table
query = "SELECT COUNT(*) FROM WDI;"
cursor = conn.cursor()
cursor.execute(query)

# Fetch the result
row_count = cursor.fetchone()[0]

# Print the result
print(f"Number of rows in the SQLite database: {row_count}")
print(f"Number of rows in the DataFrame: {len(econ_long_df)}")

# Close the connection
conn.close()


Number of rows in the SQLite database: 20665344
Number of rows in the DataFrame: 20665344


In [41]:
import sqlite3
import pandas as pd

# Load the emdat_processed CSV into a pandas DataFrame
emdat_df = pd.read_csv('./dataset_processed/emdat_processed.csv')

# Connect to the SQLite database (project.db)
conn = sqlite3.connect('project.db')

# Rename the columns in the DataFrame to match the table schema
emdat_df.rename(columns={'DisNo.': 'DisNo', 'Disaster Group': 'DisasterGroup', 'Disaster Subgroup': 'DisasterSubgroup', 'Disaster Type': 'DisasterType', 'Disaster Subtype': 'DisasterSubtype', 'Event Name': 'EventName', 'Associated Types': 'AssociatedTypes', 'OFDA/BHA Response': 'USResponse','AID Contribution (\'000 US$)': 'AidContribution', 'Magnitude Scale': 'MagnitudeScale', 'Start Year': 'StartYear', 'Start Month': 'StartMonth', 'Start Day': 'StartDay', 'End Year': 'EndYear', 'End Month': 'EndMonth', 'End Day': 'EndDay', 'Total Deaths': 'TotalDeaths', 'No. Injured': 'NoInjured', 'No. Affected': 'NoAffected', 'No. Homeless': 'NoHomeless', 'Total Affected': 'TotalAffected', 'Reconstruction Costs (\'000 US$)': 'ReconstructionCosts', 'Reconstruction Costs, Adjusted (\'000 US$)': 'ReconstructionCostsAdjusted', 'Insured Damage (\'000 US$)': 'InsuredDamage', 'Insured Damage, Adjusted (\'000 US$)': 'InsuredDamageAdjusted', 'Total Damage (\'000 US$)': 'TotalDamage', 'Total Damage, Adjusted (\'000 US$)': 'TotalDamageAdjusted'}, inplace=True)

#Drop the 'Indicator Code' column since it's not part of the SQLite schema
emdat_df.drop(columns=['Historic', 'Classification Key', 'External IDs', 'Subregion', 'Region', 'Location', 'Origin', 'Latitude', 'Longitude', 'River Basin', 'Admin Units', 'Entry Date', 'Last Update'], inplace=True)

# Drop the DISASTERS table if it exists
conn.execute("DROP TABLE IF EXISTS DISASTERS;")

# Create the DISASTERS table
create_disasters_table = """
CREATE TABLE IF NOT EXISTS DISASTERS (
    DisNo TEXT PRIMARY KEY,
    Year INTEGER,
    ISO TEXT,
    Country TEXT,
    DisasterGroup TEXT,
    DisasterSubgroup TEXT,
    DisasterType TEXT,
    DisasterSubtype TEXT,
    EventName TEXT, 
    AssociatedTypes TEXT,
    USResponse TEXT,
    Appeal TEXT,
    Declaration TEXT, 
    AidContribution TEXT,
    Magnitude REAL,
    MagnitudeScale TEXT, 
    StartYear INTEGER,
    StartMonth INTEGER,
    StartDay INTEGER,
    EndYear INTEGER,
    EndMonth INTEGER,
    EndDay INTEGER,
    TotalDeaths REAL,
    NoInjured REAL,
    NoAffected REAL,
    NoHomeless REAL, 
    TotalAffected REAL,
    ReconstructionCosts REAL,
    ReconstructionCostsAdjusted REAL,
    InsuredDamage REAL, 
    InsuredDamageAdjusted REAL,
    TotalDamage REAL, 
    TotalDamageAdjusted REAL,
    CPI REAL
);
"""
conn.execute(create_disasters_table)

# Insert the data from the DataFrame into the DISASTERS table
emdat_df.to_sql('DISASTERS', conn, if_exists='append', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data successfully inserted into the DISASTERS table in SQLite!")

Data successfully inserted into the DISASTERS table in SQLite!


In [42]:
print(f"Number of columns in emdat_df: {emdat_df.shape[1]}")
print(f"Number of rows in emdat_df: {emdat_df.shape[0]}")

Number of columns in emdat_df: 34
Number of rows in emdat_df: 25529


In [44]:
#Count the number of columns in the DISASTERS table

conn = sqlite3.connect('project.db')


query_columns = "PRAGMA table_info(DISASTERS);"
columns_info = conn.execute(query_columns).fetchall()
num_columns = len(columns_info)
print(f"Number of columns in DISASTERS table: {num_columns}")

# Count the number of rows in the DISASTERS table
query_rows = "SELECT COUNT(*) FROM DISASTERS;"
num_rows = conn.execute(query_rows).fetchone()[0]
print(f"Number of rows in DISASTERS table: {num_rows}")

conn.close()


Number of columns in DISASTERS table: 34
Number of rows in DISASTERS table: 25529
